 Mount Drive



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install texthero

In [ ]:
pip install tweet-preprocessor

In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
#reading the data
data= pd.read_csv('/modified_hasoc_task21.csv')
train_df,test_df=train_test_split(data,test_size=0.2)
#test_df= pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')   TEST DATA TO BE PROVIDED
print(train_df.columns,train_df.shape)
target_col= train_df.columns[2:]
feature_col= train_df.columns[1:2]
train_df.head()

Index(['_id', 'text', 'hate', 'offen', 'prof', 'none'], dtype='object') (3074, 6)


,_id,text,hate,offen,prof,none
1653,60c5d6bf5659ea5e55deff30,Be pussy they whole life but be macho man rand...,0,0,1,0
2732,60c5d6bf5659ea5e55def9d2,We never ever accept such kind of anti islamic...,0,0,0,1
791,60c5d6bf5659ea5e55def9d8,The National Carrier blamed &amp; shamed. Pilo...,0,0,0,1
585,60c5d6bf5659ea5e55defb3f,/3 @Tejasvi_Surya you haven't learnt even from...,0,1,0,0
2936,60c5d6bf5659ea5e55defb88,This isn't as cool as #ResignModi #BengalBurn...,0,0,0,1


In [ ]:
 target_col,feature_col

(Index(['hate', 'offen', 'prof', 'none'], dtype='object'),
 Index(['text'], dtype='object'))

In [ ]:
test_df.head()

,_id,text,hate,offen,prof,none
1092,60c5d6bf5659ea5e55defc89,@PMOIndia @narendramodi @POTUS @JoeBiden @POTU...,0,1,0,0
2154,60c5d6bf5659ea5e55def1d6,@vinodkapri @narendramodi Horrible scene....Wi...,1,0,0,0
826,60c5d6bf5659ea5e55df0311,Happy pride month from americaâ€™s whore,0,0,1,0
3652,60c5d6bf5659ea5e55def260,@BukayoSaka87 @R_K_Hoy @England Stay safe king...,0,0,1,0
672,60c5d6bf5659ea5e55deff39,"@RameshA19919981 @Tweet2_Genda Really, If this...",1,0,0,0


In [ ]:
import re 
import nltk
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer 
from textblob import TextBlob,Word
from nltk.corpus import words
nltk.download('words')
nltk.download('wordnet')
import texthero as hero
import re
from texthero import stopwords

from nltk.corpus import wordnet

import tensorflow as tf

from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

import tensorflow as tf

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def lemma_per_pos(sent):
    '''function to lemmatize according to part of speech tag'''
    tweet_tokenizer=TweetTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    lemmatized_list = [lemmatizer.lemmatize(w) for w in  tweet_tokenizer.tokenize(sent)]
    return " ".join(lemmatized_list)

def df_preprocessing(df,feature_col):
    '''
    Preprocessing of dataframe
    '''
    stop = set(stopwords.words('english'))
    df[feature_col]= (df[feature_col].pipe(hero.lowercase).
                      pipe(hero.remove_urls).
                      pipe(hero.remove_digits).
                      pipe(hero.remove_punctuation).
                      pipe(hero.remove_html_tags) )
    # lemmatization
    df[feature_col]= [lemma_per_pos(sent) for sent in df[feature_col]]
    #df[col_name]= hero.remove_stopwords(df[col_name],custom_stopwords)
    return df

In [ ]:
with tf.device('/GPU:0'): 
 proc_train_df= df_preprocessing(train_df,feature_col[0])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
proc_test_df = df_preprocessing(test_df,feature_col[0])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
pip install transformers

In [ ]:
pip install tensorflow_addons

In [ ]:
from transformers import AutoTokenizer,TFDistilBertModel, DistilBertConfig
from transformers import TFAutoModel
import tensorflow as tf 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from transformers import AdamW, get_linear_schedule_with_warmup
import tensorflow_addons as tfa

In [ ]:
#Creating tokenizer
def create_tokenizer(pretrained_weights='distilbert-base-uncased'):
  '''Function to create the tokenizer'''

  tokenizer = AutoTokenizer.from_pretrained(pretrained_weights)
  return tokenizer

#Tokenization of the data
def data_tokenization(dataset,feature_col,max_len,tokenizer):
    '''dataset: Pandas dataframe with feature name is column name 
    Pretrained_weights: selected model 
    RETURN: [input_ids, attention_mask]'''

    tokens = dataset[feature_col].apply(lambda x: tokenizer(x,return_tensors='tf', 
                                                            truncation=True,
                                                            padding='max_length',
                                                            max_length=max_len, 
                                                            add_special_tokens=True))
    input_ids= []
    attention_mask=[]
    for item in tokens:
        input_ids.append(item['input_ids'])
        attention_mask.append(item['attention_mask'])
    input_ids, attention_mask=np.squeeze(input_ids), np.squeeze(attention_mask)


    return [input_ids,attention_mask]

def bert_model(pretrained_weights,max_len,learning_rate):
  '''BERT model creation with pretrained weights
  INPUT:
  pretrained_weights: Language model pretrained weights
  max_len: input length '''
  print('Model selected:', pretrained_weights)
  bert=TFAutoModel.from_pretrained(pretrained_weights)
  
  # This is must if you would like to train the layers of language models too.
  for layer in bert.layers:
      layer.trainable = True

  # parameter declaration
#   step = tf.Variable(0, trainable=False)
#   schedule = tf.optimizers.schedules.PiecewiseConstantDecay([10000, 15000], [2e-0, 2e-1, 1e-2])
#   # lr and wd can be a function or a tensor
#   lr = learning_rate * schedule(step)
#   wd = lambda:lr * schedule(step)
#   optimizer = tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd)

  optimizer= tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
#   optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)

  # declaring inputs, BERT take input_ids and attention_mask as input
  input_ids= Input(shape=(max_len,),dtype=tf.int32,name='input_ids')
  attention_mask=Input(shape=(max_len,),dtype=tf.int32,name='attention_mask')

  bert= bert(input_ids,attention_mask=attention_mask)
  x= bert[0][:,0,:]
  x=tf.keras.layers.Dropout(0.1)(x)
  x=tf.keras.layers.Dense(64)(x)
  x=tf.keras.layers.Dense(32)(x)

  output=tf.keras.layers.Dense(4,activation='sigmoid')(x)

  model=Model(inputs=[input_ids,attention_mask],outputs=[output])
  # compiling model 
  model.compile(optimizer=optimizer,
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE,name='binary_crossentropy'),
                metrics=['accuracy'])
  return model

In [ ]:
pretrained_weights='bert-base-uncased'
max_len=256
epochs=5
learning_rate=2e-5
batch_size=4

In [ ]:
tokenizer= create_tokenizer(pretrained_weights)

In [ ]:
x_train= data_tokenization(proc_train_df,feature_col[0],max_len,tokenizer)

In [ ]:
x_train

[array([[  101,  2022, 22418, ...,     0,     0,     0],
        [  101,  2057,  2196, ...,     0,     0,     0],
        [  101,  1996,  2120, ...,     0,     0,     0],
        ...,
        [  101,  2111,  2024, ...,     0,     0,     0],
        [  101, 19734,  8883, ...,     0,     0,     0],
        [  101,  1996,  3308, ...,     0,     0,     0]], dtype=int32),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)]

In [ ]:
y_train= proc_train_df[target_col].values
y_train

array([[0, 0, 1, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       ...,
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

In [ ]:
y_train.shape

(3074, 4)

In [ ]:
bert=bert_model(pretrained_weights,max_len,learning_rate)
bert.summary()

Model selected: bert-base-uncased


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [ ]:
with tf.device('/GPU:0'):
    bert.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_split=0.1)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/keras/backend.py:4994: UserWarning:

"`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"



692/692 [==============================] - 376s 514ms/step - loss: 0.4234 - accuracy: 0.5824 - val_loss: 0.3730 - val_accuracy: 0.6494
Epoch 2/5
692/692 [==============================] - 353s 510ms/step - loss: 0.3048 - accuracy: 0.7314 - val_loss: 0.3217 - val_accuracy: 0.7240
Epoch 3/5
692/692 [==============================] - 344s 497ms/step - loss: 0.1814 - accuracy: 0.8539 - val_loss: 0.4430 - val_accuracy: 0.7013
Epoch 4/5
692/692 [==============================] - 344s 497ms/step - loss: 0.0945 - accuracy: 0.9320 - val_loss: 0.5384 - val_accuracy: 0.6234
Epoch 5/5
692/692 [==============================] - 352s 509ms/step - loss: 0.0521 - accuracy: 0.9657 - val_loss: 0.6005 - val_accuracy: 0.6721


In [ ]:
test_ids= test_df['_id']
x_test= data_tokenization(proc_test_df,feature_col[0],max_len,tokenizer)
x_test

[array([[  101,  7610, 28765, ...,     0,     0,     0],
        [  101, 19354,  7716, ...,     0,     0,     0],
        [  101,  3407,  6620, ...,     0,     0,     0],
        ...,
        [  101,  2279, 12731, ...,     0,     0,     0],
        [  101,  4931,  2009, ...,     0,     0,     0],
        [  101,  2848, 16901, ...,     0,     0,     0]], dtype=int32),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)]

In [ ]:
test_df['_id']

1092    60c5d6bf5659ea5e55defc89
2154    60c5d6bf5659ea5e55def1d6
826     60c5d6bf5659ea5e55df0311
3652    60c5d6bf5659ea5e55def260
672     60c5d6bf5659ea5e55deff39
                  ...           
1771    60c5d6bf5659ea5e55def922
2816    60c5d6bf5659ea5e55df02d3
3193    60c5d6bf5659ea5e55df024c
3377    60c5d6bf5659ea5e55df026a
3641    60c5d6bf5659ea5e55def6b6
Name: _id, Length: 769, dtype: object

In [ ]:
preds= bert.predict(x_test)



In [ ]:
submiss_df= pd.DataFrame(preds, columns= target_col)
submiss_df['id']=list(test_df['_id'])
submiss_df

,hate,offen,prof,none,id
0,0.002043,0.974148,0.001425,0.052254,60c5d6bf5659ea5e55defc89
1,0.001626,0.979116,0.009639,0.011772,60c5d6bf5659ea5e55def1d6
2,0.002839,0.000263,0.995727,0.001205,60c5d6bf5659ea5e55df0311
3,0.000087,0.000179,0.998941,0.000149,60c5d6bf5659ea5e55def260
4,0.007449,0.000262,0.001037,0.992124,60c5d6bf5659ea5e55deff39
...,...,...,...,...,...
764,0.009057,0.601427,0.269961,0.016167,60c5d6bf5659ea5e55def922
765,0.049448,0.001282,0.002352,0.982864,60c5d6bf5659ea5e55df02d3
766,0.000408,0.000688,0.994915,0.000149,60c5d6bf5659ea5e55df024c
767,0.000066,0.001221,0.993228,0.000038,60c5d6bf5659ea5e55df026a


In [ ]:
submiss_df = submiss_df[['id']+target_col.tolist()]
submiss_df

,id,hate,offen,prof,none
0,60c5d6bf5659ea5e55defc89,0.002043,0.974148,0.001425,0.052254
1,60c5d6bf5659ea5e55def1d6,0.001626,0.979116,0.009639,0.011772
2,60c5d6bf5659ea5e55df0311,0.002839,0.000263,0.995727,0.001205
3,60c5d6bf5659ea5e55def260,0.000087,0.000179,0.998941,0.000149
4,60c5d6bf5659ea5e55deff39,0.007449,0.000262,0.001037,0.992124
...,...,...,...,...,...
764,60c5d6bf5659ea5e55def922,0.009057,0.601427,0.269961,0.016167
765,60c5d6bf5659ea5e55df02d3,0.049448,0.001282,0.002352,0.982864
766,60c5d6bf5659ea5e55df024c,0.000408,0.000688,0.994915,0.000149
767,60c5d6bf5659ea5e55df026a,0.000066,0.001221,0.993228,0.000038


#task_2

In [ ]:
column_names = ["id", "label"]
submission_df = pd.DataFrame(columns = column_names)
submission_df

,id,label


In [ ]:
for i in range(len(submiss_df)):
  temp = max(submiss_df['hate'].iloc[i],submiss_df['offen'].iloc[i],submiss_df['none'].iloc[i],submiss_df['prof'].iloc[i])
  id_temp = submiss_df['id'].iloc[i]
  if temp == submiss_df['hate'].iloc[i]:
    submission_df.loc[i] = [id_temp,'HATE']
  elif temp == submiss_df['offen'].iloc[i]:
    submission_df.loc[i] = [id_temp,'OFFN']
  elif temp == submiss_df['prof'].iloc[i]:
    submission_df.loc[i] = [id_temp,'PRFN']
  elif temp == submiss_df['none'].iloc[i]:
    submission_df.loc[i] = [id_temp,'NONE']


In [ ]:
submission_df

,id,label
0,60c5d6bf5659ea5e55defc89,OFFN
1,60c5d6bf5659ea5e55def1d6,OFFN
2,60c5d6bf5659ea5e55df0311,PRFN
3,60c5d6bf5659ea5e55def260,PRFN
4,60c5d6bf5659ea5e55deff39,NONE
...,...,...
764,60c5d6bf5659ea5e55def922,OFFN
765,60c5d6bf5659ea5e55df02d3,NONE
766,60c5d6bf5659ea5e55df024c,PRFN
767,60c5d6bf5659ea5e55df026a,PRFN


In [ ]:
test_df

,_id,text,hate,offen,prof,none
1092,60c5d6bf5659ea5e55defc89,pmoindia narendramodi potus joebiden potus vp ...,0,1,0,0
2154,60c5d6bf5659ea5e55def1d6,vinodkapri narendramodi horrible scene will mo...,1,0,0,0
826,60c5d6bf5659ea5e55df0311,happy pride month from americaâ € ™ s whore,0,0,1,0
3652,60c5d6bf5659ea5e55def260,bukayosaka 87 r k hoy england stay safe king d...,0,0,1,0
672,60c5d6bf5659ea5e55deff39,ramesha 19919981 tweet 2 genda really if this ...,1,0,0,0
...,...,...,...,...,...,...
1771,60c5d6bf5659ea5e55def922,sebi 3525 discord i d agree with you but shut ...,0,0,1,0
2816,60c5d6bf5659ea5e55df02d3,hoseok literally created the lgbt community yo...,0,0,1,0
3193,60c5d6bf5659ea5e55df024c,next cunt challenge me in the middle lane with...,0,0,1,0
3377,60c5d6bf5659ea5e55df026a,hey it zero urmomavery 413 at least i got a jo...,0,0,1,0


In [ ]:
idx = []
for i in range(len(test_df)):
  idx.append(i)

In [ ]:
test_df = test_df.set_index(pd.Index(idx))

In [ ]:
test_df

,_id,text,hate,offen,prof,none
0,60c5d6bf5659ea5e55defc89,pmoindia narendramodi potus joebiden potus vp ...,0,1,0,0
1,60c5d6bf5659ea5e55def1d6,vinodkapri narendramodi horrible scene will mo...,1,0,0,0
2,60c5d6bf5659ea5e55df0311,happy pride month from americaâ € ™ s whore,0,0,1,0
3,60c5d6bf5659ea5e55def260,bukayosaka 87 r k hoy england stay safe king d...,0,0,1,0
4,60c5d6bf5659ea5e55deff39,ramesha 19919981 tweet 2 genda really if this ...,1,0,0,0
...,...,...,...,...,...,...
764,60c5d6bf5659ea5e55def922,sebi 3525 discord i d agree with you but shut ...,0,0,1,0
765,60c5d6bf5659ea5e55df02d3,hoseok literally created the lgbt community yo...,0,0,1,0
766,60c5d6bf5659ea5e55df024c,next cunt challenge me in the middle lane with...,0,0,1,0
767,60c5d6bf5659ea5e55df026a,hey it zero urmomavery 413 at least i got a jo...,0,0,1,0


#analyse

In [ ]:
column_names = ["id","hate","offen","prof","none"]
analyse_df = pd.DataFrame(columns = column_names)
analyse_df

,id,hate,offen,prof,none


In [ ]:
for i in range(len(submission_df)):
  id_temp = submission_df['id'].iloc[i]
  if submission_df['label'].iloc[i] == "NONE":
    analyse_df.loc[i] = [id_temp,0,0,0,1]
  elif submission_df['label'].iloc[i] == "PRFN":
    analyse_df.loc[i] = [id_temp,0,0,1,0]
  elif submission_df['label'].iloc[i] == "OFFN":
    analyse_df.loc[i] = [id_temp,0,1,0,0]
  elif submission_df['label'].iloc[i] == "HATE":
    analyse_df.loc[i] = [id_temp,1,0,0,0]

In [ ]:
analyse_df

,id,hate,offen,prof,none
0,60c5d6bf5659ea5e55defc89,0,1,0,0
1,60c5d6bf5659ea5e55def1d6,0,1,0,0
2,60c5d6bf5659ea5e55df0311,0,0,1,0
3,60c5d6bf5659ea5e55def260,0,0,1,0
4,60c5d6bf5659ea5e55deff39,0,0,0,1
...,...,...,...,...,...
764,60c5d6bf5659ea5e55def922,0,1,0,0
765,60c5d6bf5659ea5e55df02d3,0,0,0,1
766,60c5d6bf5659ea5e55df024c,0,0,1,0
767,60c5d6bf5659ea5e55df026a,0,0,1,0


In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix

In [ ]:
(analyse_df['hate'] == test_df['hate']).value_counts()

True     648
False    121
Name: hate, dtype: int64

In [ ]:
acc = 628/(628+141)
acc

0.8166449934980494

In [ ]:
(analyse_df['offen'] == test_df['offen']).value_counts()


True     651
False    118
Name: offen, dtype: int64

In [ ]:
acc_o = 636/(628+141)
acc_o

0.8270481144343304

In [ ]:
(analyse_df['prof'] == test_df['prof']).value_counts()

True     638
False    131
Name: prof, dtype: int64

In [ ]:
acc_p = 653/(628+141)
acc_p

0.8491547464239272

In [ ]:
(analyse_df['none'] == test_df['none']).value_counts()

True     603
False    166
Name: none, dtype: int64

In [ ]:
acc_n = 613/(628+141)
acc_n

0.7971391417425228

In [ ]:
column_names = ["id", "label"]
testcomp_df = pd.DataFrame(columns = column_names)
testcomp_df

,id,label


In [ ]:
for i in range(len(test_df)):
  temp = max(test_df['hate'].iloc[i],test_df['offen'].iloc[i],test_df['none'].iloc[i],test_df['prof'].iloc[i])
  id_temp = test_df['_id'].iloc[i]
  if temp == test_df['hate'].iloc[i]:
    testcomp_df.loc[i] = [id_temp,'HATE']
  elif temp == test_df['offen'].iloc[i]:
    testcomp_df.loc[i] = [id_temp,'OFFN']
  elif temp == test_df['prof'].iloc[i]:
    testcomp_df.loc[i] = [id_temp,'PRFN']
  elif temp == test_df['none'].iloc[i]:
    testcomp_df.loc[i] = [id_temp,'NONE']

In [ ]:
testcomp_df

,id,label
0,60c5d6bf5659ea5e55defc89,OFFN
1,60c5d6bf5659ea5e55def1d6,HATE
2,60c5d6bf5659ea5e55df0311,PRFN
3,60c5d6bf5659ea5e55def260,PRFN
4,60c5d6bf5659ea5e55deff39,HATE
...,...,...
764,60c5d6bf5659ea5e55def922,PRFN
765,60c5d6bf5659ea5e55df02d3,PRFN
766,60c5d6bf5659ea5e55df024c,PRFN
767,60c5d6bf5659ea5e55df026a,PRFN


In [ ]:
(testcomp_df['label'] == submission_df['label']).value_counts()

True     501
False    268
Name: label, dtype: int64

In [ ]:
acc = 501/(501+268)